## CSCI 5654 - Linear Programming - Project

***
#### Team Members
#### 1. Ketan Ramesh
#### 2. Shreyas Gopalakrishna
***

## Vehicle Routing Problem

In [1]:
!pip3 install pulp

     |████████████████████████████████| 40.6MB 71kB/s 


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import pulp
import gurobipy as gp
from gurobipy import GRB

## Capacity Vehicle Routing Problem - Model 2

Paper Link - [Integer linear programming formulation for a vehicle routing problem](https://doi.org/10.1016/0377-2217(91)90338-V)

<div align="justify">
The code provided below creates a <code>class</code> to frame and solve the Capacity Vehicle Routing Problem (CVRP) using a Vehicle Path variant of the <b>Kulkarni-Bhave formulation</b> as mentioned in the paper. The problem as a Mixed ILP is as follows:
</div>

$$
min \sum_{(i, j) \in A} c_{ij} x_{ij}
$$

$$
\sum_{i=1, i \neq j}^{N} x_{ij} = 1, j=1, ..., N-1, - (2.1)
$$

$$
\sum_{j=1, j \neq i}^{N-1} x_{ij} +x_{iN^{'}} = 1, j=1, ..., N-1, - (2.2)
$$

$$
\sum_{j=1}^{N-1} x_{Nj} = V, - (2.3)
$$

$$
\sum_{i=1}^{N-1} x_{iN^{'}} = V, - (2.4)
$$

$$
y_i - y_j +(L+1)x_{ij} \leq L, \forall (i, j) \in A, - (2.5)
$$

$$
y_{N^{'}} - y_N \leq L, \forall (i, j) \in A, - (2.6)
$$

$$
u_i - u_j + Wx_{ij} \leq W - Q_j, \forall (i, j) \in A, - (2.7)
$$

$$
u_{N^{'}} - u_N = W,  - (2.8)
$$

$$
v_i - v_j + (c_{ij} + T) x_{ij} \leq T, \forall (i, j) \in A, - (2.9)
$$

$$
v_{N^{'}} - v_N = T,  - (2.10)
$$

$$
x_{ij} = \{0, 1\} , \forall (i, j) \in A
$$

where,

N - 1 customer locations are denoted as 1, 2, ..., N - 1. The location of depot is denoted by N.<br />
Q<sub>j</sub> = Demand of customer at location j.<br />
V = Number of delivery vehicles.<br />
L = Number of customers that a vehicle can service in a route (preset).<br />
T = Maximum distance that any route can reach (preset).<br />
W = Capacity of the vehicle.<br />
c<sub>ij</sub> = Cost incurred on traveling from i to j.<br />
Depot = N, N<sup>'</sup><br />

$$
A = \{(N, i), (i, j), (i, N^{'}) : 1 \leq i \neq j \leq N - 1\},
$$

<b>Decision Variables:</b>
1. x<sub>ij</sub> = {<b>1</b>, if edge (i, j) present in any route. <b>0</b>, otherwise}. 

2. y<sub>ij</sub>, u<sub>ij</sub> and v<sub>ij</sub> = A set of continuous non-negative variables.

<div align="justify">
The objective is to minimize the sum of the distances of all routes that satisfy the given constraints. The constraint (2, 1) ensures that all customer nodes are visited by exactly once while the constraint (2, 2) ensures that the number of paths arising from a node is exactly one (prevents multiple visits to the same node). The constraints (2, 3). and (2, 4) add the requirement that all vehicles be used for routing. The constraints (2, 5) and (2, 6) are used to eliminate sub tours in the problem. The constraints (2, 7) and (2, 8) ensure that the cumulative load on the route doesn't exceed the vehicle capacity. The constraints (2, 9) and (2, 10) ensure that each vehicle route doesn't exceed a set distance.
</div>

In [1]:
class CVRP2_GUROBI:
		def __init__(self, numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix, A, L, T):
				self.numberOfCustomers = numberOfCustomers
				self.numberOfVehicles  = numberOfVehicles
				self.capacityOfVehicle = capacityOfVehicle
				self.demandOfCustomers = demandOfCustomers
				self.costMatrix        = costMatrix
				self.A = A
				self.L = L
				self.T = T
				self.N = len(self.costMatrix)
				self.initializeLP()

		def initializeLP(self):
				self.cvrpLP = gp.Model("CVRP_2")
				objective = None
				x, y = [], []
				u, v = [], []

				# Decision Variables
				for i in range(self.N):
					xRow = []
					y.append(self.cvrpLP.addVar(name='y('+str(i)+')', vtype=GRB.CONTINUOUS, lb=0))
					u.append(self.cvrpLP.addVar(name='u('+str(i)+')', vtype=GRB.CONTINUOUS, lb=0))
					v.append(self.cvrpLP.addVar(name='v('+str(i)+')', vtype=GRB.CONTINUOUS, lb=0))
					for j in range(self.N):
						xRow.append(self.cvrpLP.addVar(name='x('+str(i)+','+str(j)+')', vtype=GRB.BINARY, lb=0, ub=1))
					x.append(xRow)

				# Adding objective
				for tup in self.A:
					objective += costMatrix[tup[0]][tup[1]] * x[tup[0]][tup[1]]
				self.cvrpLP.setObjective(objective,GRB.MINIMIZE)

				# Adding constraint 1
				for j in range(1, self.N - 1):
					constraint1 = None
					for i in range(1, self.N):
						if (i != j):
							if(constraint1 == None):
								constraint1 = x[i][j]
							else:
								constraint1 = constraint1 + x[i][j]
					self.cvrpLP.addConstr(constraint1 == 1)

				# Adding constraint 2
				for i in range(1, self.N - 1):
					constraint2 = None
					for j in range(1, self.N - 1):
						if (i != j):
							if(constraint2 == None):
								constraint2 = x[i][j]
							else:
								constraint2 = constraint2 + x[i][j]
					constraint2 = constraint2 + x[i][self.N - 1]
					self.cvrpLP.addConstr(constraint2 == 1)

				# Adding constraint 3
				constraint3 = None
				for i in range(1, self.N - 1):
					if(constraint3 == None):
						constraint3 = x[0][i]
					else:
						constraint3 = constraint3 + x[0][i]
				self.cvrpLP.addConstr(constraint3 == self.numberOfVehicles)

				# Adding constraint 4
				constraint4 = None
				for i in range(1, self.N - 1):
					if(constraint4 == None):
						constraint4 = x[i][self.N - 1]
					else:
						constraint4 = constraint4 + x[i][self.N - 1]
				self.cvrpLP.addConstr(constraint4 == self.numberOfVehicles)

				# Adding constraint 5
				for tup in self.A:
					self.cvrpLP.addConstr(y[tup[0]] - y[tup[1]] + (self.L + 1) * x[tup[0]][tup[1]] <= self.L)

				# Adding constraint 6
				self.cvrpLP.addConstr(y[-1] - y[0] <= self.L + 1)

				# Adding constraint 7
				for i, j in self.A:
					self.cvrpLP.addConstr(u[i] - u[j] + self.capacityOfVehicle * x[i][j] <= self.capacityOfVehicle - self.demandOfCustomers[j])

				# Adding constraint 8
				self.cvrpLP.addConstr( u[-1] - u[0] == self.capacityOfVehicle)

				# Adding constraint 9
				for i, j in self.A:
					self.cvrpLP.addConstr(v[i] - v[j] + (self.costMatrix[i][j] + self.T) * x[i][j] <= self.T)
				
				# Adding constrint 10
				self.cvrpLP.addConstr(v[-1] - v[0] == self.T)

				# Adding flow constraint
				for h in range(1, len(costMatrix)-1):
						constraint5a, constraint5b = None, None
						for i in range(0, len(costMatrix)-1):
								if(i != h):
									if(constraint5a == None):
										constraint5a = x[i][h]
									else:
										constraint5a = constraint5a + x[i][h]
						for j in range(1, len(costMatrix)):
								if(j != h):
									if(constraint5b == None):
										constraint5b = x[h][j]
									else:
										constraint5b = constraint5b + x[h][j]
						self.cvrpLP.addConstr(constraint5a - constraint5b == 0)

		def solve(self):
				status = self.cvrpLP.optimize()
				print(status)
		
		def getResult(self):
				print("Objective value: ", self.cvrpLP.ObjVal)
				for v in self.cvrpLP.getVars():
						print(v.varName, " = ", v.x)
				return self.cvrpLP

## Random Testing

In [4]:
numberOfCustomers = 5
capacityOfVehicle = 10
numberOfVehicles = 3
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {1: 1, 2: 5, 3: 8, 4: 5, 5: 5, 6: 0, 0: 0}
# demandOfCustomers[0] = 0
# demandOfCustomers[numberOfCustomers+1] = 0

# Creating random coordinates
xCoordinates = [30, 20, 10, 10, 40, 50, 30]
yCoordinates = [30, 40, 45, 30, 10, 30, 30]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 0
costMatrix

array([[ 0., 14., 25., 20., 22., 20.,  0.],
       [14.,  0., 11., 14., 36., 31., 14.],
       [25., 11.,  0., 15., 46., 42., 25.],
       [20., 14., 15.,  0., 36., 40., 20.],
       [22., 36., 46., 36.,  0., 22., 22.],
       [20., 31., 42., 40., 22.,  0., 20.],
       [ 0., 14., 25., 20., 22., 20.,  0.]])

In [5]:
A = []
L = numberOfCustomers
T = 999999
for i in range(1, len(costMatrix) - 1):
  A.append((0, i))
  for j in range(1, len(costMatrix) - 1):
    if (i != j):
      A.append((i, j))
  A.append((i, len(costMatrix) - 1))
print(len(A))

30


In [8]:
cvrp = CVRP2_GUROBI(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix, A, L, T)

Using license file /Users/shreyas/gurobi.lic
Academic license - for non-commercial use only


In [9]:
cvrp.solve()
result = cvrp.getResult()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 110 rows, 70 columns and 386 nonzeros
Model fingerprint: 0x6b9ab9d9
Variable types: 21 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 162.0000000
Presolve removed 8 rows and 22 columns
Presolve time: 0.00s
Presolved: 102 rows, 48 columns, 330 nonzeros
Variable types: 18 continuous, 30 integer (30 binary)

Root relaxation: objective 1.508000e+02, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  150.80000    0    6  162.00000  150.80000  6.91%     -    0s
H    0     0                     154.0000000  150.80000  2.08%     -    0s
     0     0     cutoff    0       154.00000  154.00000  0.00

In [16]:
# Use output to get path
# gets variables as (x,y) coordinates
variables = []
for v in result.getVars():
    if('x' in v.varName and v.x == 1):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
        variables.append((int(temp[0]),int(temp[1])))
print(variables)

# recursive calls for getting the path
def recursiveList(start, L, X, c):
    if start in c:
        return X
    for item in L:
        if(item[0] == start):
            X.append(item)
            c.append(start)
            return recursiveList(item[1], L, X, c)
    return X

pathList = []
setList = []
start = 0
for v in variables:
    if(v[0] == start):
        path = recursiveList(v[1], variables, [v], [v[0]])
        print(path)
        pathList.append(path)
        set1 = []
        for i in path:
            set1.append(i[0])
            set1.append(i[1])
        setList.append(list(set(set1)))

[(0, 1), (0, 3), (0, 5), (1, 2), (2, 6), (3, 6), (4, 6), (5, 4), (6, 1), (6, 3), (6, 5)]


In [11]:
xCoordinates = [30, 20, 10, 10, 40, 50, 35]
yCoordinates = [30, 40, 45, 30, 10, 30, 35]

coordinateList = []
for s in variables:
    coordinate = []
    for j in s:
        coordinate.append([float(xCoordinates[j]), float(yCoordinates[j])])
    coordinateList.append(coordinate)
# print(coordinateList[0])

In [15]:
def addToPlot(L):
    x_val = [x[0] for x in L]
    y_val = [x[1] for x in L]

    r = random.random()
    b = random.random()
    g = random.random()
    newColor = (r, g, b)
    
    plt.scatter(x_val,y_val, c=newColor, edgecolor='black', linewidth=2, s=1000, marker='h')
    ax = plt.axes()
    
    length = len(L)-1
    
    for i in range(length):
        ax.arrow(L[i][0],  #x1
                    L[i][1],  # y1
                    L[i+1][0]-L[i][0], # x2 - x1
                    L[i+1][1]-L[i][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
    
    # ax.arrow(L[-1][0],  #x1
    #                 L[-1][1],  # y1
    #                 L[0][0]-L[-1][0], # x2 - x1
    #                 L[0][1]-L[-1][1], # y2 - y1
    #                 width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
    #                 )
for i in range(len(coordinateList)):
    addToPlot(coordinateList[i])
plt.scatter(coordinateList[0][0][0],coordinateList[0][0][1], c='black', edgecolor='black', linewidth='2', s=1000, marker='h')
plt.show()



## TESTING WITH PRE_DEFINED DATA N=15
## P-n16-k8.vrp - Augerat et al. Set - P

In [17]:
numberOfCustomers = 15
capacityOfVehicle = 35
numberOfVehicles = 8
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {0:0,1:19,2:30,3:16,4:23,5:11,6:31,7:15,8:28,9:8,10:8,11:7,12:14,13:6,14:19,15:11}
# demandOfCustomers[0] = 0
demandOfCustomers[numberOfCustomers+1] = 0

In [18]:
xCoordinates = [30,37,49,52,31,52,42,52,57,62,42,27,43,58,58,37,30]
yCoordinates = [40,52,49,64,62,33,41,41,58,42,57,68,67,48,27,69,40]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 999999

In [19]:
A = []
L = numberOfCustomers
T = 999999
for i in range(1, len(costMatrix) - 1):
  A.append((0, i))
  for j in range(1, len(costMatrix) - 1):
    if (i != j):
      A.append((i, j))
  A.append((i, len(costMatrix) - 1))
print(len(A))

240


In [20]:
cvrp2 = CVRP2_GUROBI(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix, A, L, T)

In [21]:
cvrp2.solve()
result = cvrp2.getResult()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 770 rows, 340 columns and 3096 nonzeros
Model fingerprint: 0x4f6e72e5
Variable types: 51 continuous, 289 integer (289 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [6e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 18 rows and 52 columns
Presolve time: 0.01s
Presolved: 752 rows, 288 columns, 2640 nonzeros
Variable types: 48 continuous, 240 integer (240 binary)
Found heuristic solution: objective 560.0000000

Root relaxation: objective 3.994286e+02, 109 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  399.42857    0   23  560.00000  399.42857  28.7%     -    0s
     0     0  400.49524    0   27  560.00000  400.49524  28.5%     -    0s
     0     0  402.20952    0   27  560.00000  402.2

## TESTING WITH PRE_DEFINED DATA N=20
## P-n20-k2.vrp - Augerat et al. Set - P

In [27]:
numberOfCustomers = 19
capacityOfVehicle = 160
numberOfVehicles = 2
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = [0, 19, 30, 16, 23, 11, 31, 15, 28, 8, 8, 7, 14, 6, 19, 11, 26, 17, 6, 15, 0]
# demandOfCustomers[0] = 0
# demandOfCustomers[numberOfCustomers+1] = 0

xCoordinates = [30, 37, 49, 52, 31, 52, 42, 52, 57, 62, 42, 27, 43, 58, 58, 37, 61, 62, 63, 45, 30]
yCoordinates = [40, 52, 49, 64, 62, 33, 41, 41, 58, 42, 57, 68, 67, 48, 27, 69, 33, 63, 69, 35, 40]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 0
costMatrix

A = []
L = numberOfCustomers
T = 999999
for i in range(1, len(costMatrix) - 1):
  A.append((0, i))
  for j in range(1, len(costMatrix) - 1):
    if (i != j):
      A.append((i, j))
  A.append((i, len(costMatrix) - 1))
print(len(A))

380


In [28]:
cvrp2 = CVRP2_GUROBI(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix, A, L, T)
cvrp2.solve()
result = cvrp2.getResult()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1202 rows, 504 columns and 4908 nonzeros
Model fingerprint: 0x7f4d03a4
Variable types: 63 continuous, 441 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [6e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 22 rows and 64 columns
Presolve time: 0.04s
Presolved: 1180 rows, 440 columns, 4180 nonzeros
Variable types: 60 continuous, 380 integer (380 binary)

Root relaxation: objective 1.736125e+02, 134 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  173.61250    0   36          -  173.61250      -     -    0s
     0     0  190.00000    0   34          -  190.00000      -     -    0s
     0     0  190.00000    0   34          -  190.00000      -     -    0s
     0     0  190.0000

x(20,11)  =  0.0
x(20,12)  =  0.0
x(20,13)  =  0.0
x(20,14)  =  0.0
x(20,15)  =  0.0
x(20,16)  =  0.0
x(20,17)  =  0.0
x(20,18)  =  0.0
x(20,19)  =  1.0
x(20,20)  =  0.0


## TESTING WITH PRE_DEFINED DATA N=32
## A-n32-k5.vrp - Augerat et al. Set - A


In [22]:
numberOfCustomers = 31
capacityOfVehicle = 100
numberOfVehicles = 5
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {0:0,1:19,2:21,3:6,4:19,5:7,6:12,7:16,8:6,9:16,10:8,11:14,12:21,13:16,14:3,15:22,16:18,17:19,18:1,19:24,20:8,21:12,22:4,23:8,24:24,25:24,26:2,27:20,28:15,29:2,30:14,31:9}
# demandOfCustomers[0] = 0
demandOfCustomers[numberOfCustomers+1] = 0

In [23]:
xCoordinates = [82,96,50,49,13,29,58,84,14,2,3,5,98,84,61,1,88,91,19,93,50,98,5,42,61,9,80,57,23,20,85,98,82]
yCoordinates = [76,44,5,8,7,89,30,39,24,39,82,10,52,25,59,65,51,2,32,3,93,14,42,9,62,97,55,69,15,70,60,5,76]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 999999

In [24]:
A = []
L = numberOfCustomers
T = 999999
for i in range(1, len(costMatrix) - 1):
  A.append((0, i))
  for j in range(1, len(costMatrix) - 1):
    if (i != j):
      A.append((i, j))
  A.append((i, len(costMatrix) - 1))
print(len(A))

992


In [25]:
cvrp2 = CVRP2_GUROBI(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix, A, L, T)

In [26]:
cvrp2.solve()
result = cvrp2.getResult()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3074 rows, 1188 columns and 12840 nonzeros
Model fingerprint: 0x7332ad83
Variable types: 99 continuous, 1089 integer (1089 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 34 rows and 100 columns
Presolve time: 0.04s
Presolved: 3040 rows, 1088 columns, 10912 nonzeros
Variable types: 96 continuous, 992 integer (992 binary)

Root relaxation: objective 5.349900e+02, 190 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  534.99000    0   48          -  534.99000      -     -    0s
     0     0  559.31667    0   58          -  559.31667      -     -    0s
     0     0  572.61667    0   58          -  572.61667      -     -    0s
     0     0  5

 524036 383193  748.60608   77   44  847.00000  625.76434  26.1%  24.3  390s
 531630 389307  691.09143   75   33  847.00000  625.90333  26.1%  24.4  395s
 537475 393883  732.02667   74   31  847.00000  626.00000  26.1%  24.4  400s
 545048 399407     cutoff  130       847.00000  626.12000  26.1%  24.4  405s
 551237 404673  666.51333   72   42  847.00000  626.23000  26.1%  24.4  410s
 558069 409806  770.87113   84   47  847.00000  626.33333  26.1%  24.4  415s
 564619 415299     cutoff   83       847.00000  626.46000  26.0%  24.4  420s
 572530 421640  658.12071   47   48  847.00000  626.57069  26.0%  24.4  425s
 579351 426799  643.08000   47   46  847.00000  626.67667  26.0%  24.4  430s
 586686 432541  652.34743   58   53  847.00000  626.78947  26.0%  24.4  435s
 593597 437823  770.31875   59   40  847.00000  626.90417  26.0%  24.4  440s
 599972 442432  832.70500  126   32  847.00000  627.00000  26.0%  24.4  445s
 606180 448328  673.46629   65   39  847.00000  627.08333  26.0%  24.4  450s

 1218938 916795  779.83000   70   25  844.00000  633.42123  25.0%  24.7  915s
 1225449 921402  675.91994   68   44  844.00000  633.47667  24.9%  24.7  920s
 1233605 928078  758.88790   83   42  844.00000  633.52500  24.9%  24.7  925s
 1239140 932405  677.37200   59   35  844.00000  633.56833  24.9%  24.7  930s
 1245148 936664  790.06114   72   45  844.00000  633.60707  24.9%  24.7  935s
 1253173 942618  735.35971   70   30  844.00000  633.67308  24.9%  24.7  940s
 1259812 947406  689.96167   61   42  844.00000  633.72000  24.9%  24.7  945s
 1266629 952890  773.31180   60   37  844.00000  633.76503  24.9%  24.7  950s
 1272762 957075  815.47000   83   28  844.00000  633.80667  24.9%  24.7  955s
 1280297 962664  768.49600   78   37  844.00000  633.87149  24.9%  24.8  960s
 1286403 966441  797.83000   94   20  844.00000  633.93333  24.9%  24.8  966s
 1289361 968459  656.74364   63   40  844.00000  633.96315  24.9%  24.8  971s
H1289376 938098                     831.0000000  633.96315  23.7

 1745481 923949  753.77333   74   41  778.00000  637.96400  18.0%  26.6 1850s
 1750048 926412  675.41000   58   30  778.00000  638.00000  18.0%  26.7 1855s
 1754387 928142  773.99000   62   38  778.00000  638.03860  18.0%  26.7 1860s
 1757618 930546  648.56676   54   43  778.00000  638.06739  18.0%  26.7 1865s
 1762132 933176  733.66333   73   39  778.00000  638.11400  18.0%  26.7 1870s
 1766539 935436  649.94000   49   33  778.00000  638.14954  18.0%  26.7 1875s
 1770891 937739  705.63750   55   32  778.00000  638.18250  18.0%  26.7 1880s
 1776548 940840 infeasible   64       778.00000  638.23000  18.0%  26.8 1886s
 1780646 943251  672.56000   51   28  778.00000  638.25800  18.0%  26.8 1890s
 1786737 946829  697.08000   56   39  778.00000  638.31250  18.0%  26.8 1895s
 1791763 949475  677.53076   47   46  778.00000  638.35789  17.9%  26.8 1900s
 1796138 951864  651.34591   49   49  778.00000  638.40000  17.9%  26.8 1905s
 1801898 955144  764.05000   72   30  778.00000  638.44000  17.9

 2187623 1163264  687.56986   59   41  778.00000  641.17143  17.6%  28.0 2376s
 2190673 1164981  747.51333   76   21  778.00000  641.19000  17.6%  28.0 2380s
 2195370 1167574  692.99580   53   43  778.00000  641.22000  17.6%  28.1 2385s
 2200016 1170215  723.49000   58   42  778.00000  641.24615  17.6%  28.1 2390s
 2204044 1172285  755.81500   67   38  778.00000  641.26683  17.6%  28.1 2395s
 2209064 1174927  674.71000   52   38  778.00000  641.30000  17.6%  28.1 2401s
 2213765 1177505  675.69000   58   36  778.00000  641.33000  17.6%  28.1 2406s
 2216741 1179042  676.72357   49   37  778.00000  641.35010  17.6%  28.1 2410s
 2221615 1181761  771.30000   71   42  778.00000  641.38000  17.6%  28.1 2416s
 2224891 1183398  688.47714   70   38  778.00000  641.40000  17.6%  28.1 2420s
 2229635 1185750  717.18283   58   41  778.00000  641.41826  17.6%  28.1 2425s
 2233380 1187694  664.05925   55   57  778.00000  641.43750  17.6%  28.1 2430s
 2238068 1190268  699.84167   54   46  778.00000  64

...

 13915812 6485117  753.57000   65   37  773.00000  662.61500  14.3%  31.7 15785s
 13919283 6486653  767.48500   65   25  773.00000  662.61789  14.3%  31.7 15790s
 13923079 6488188  765.42500   73   36  773.00000  662.62000  14.3%  31.7 15796s
 13926786 6489720  745.05787   73   21  773.00000  662.62400  14.3%  31.7 15801s
 13930326 6491338  754.45125   70   28  773.00000  662.62667  14.3%  31.7 15806s
 13934027 6492814  760.48000   68   27  773.00000  662.63000  14.3%  31.7 15813s
 13935826 6493523  766.78823   72   41  773.00000  662.63125  14.3%  31.7 15815s
 13941309 6495854  758.87000   67   41  773.00000  662.63646  14.3%  31.7 15821s
 13944937 6497260  702.86293   63   39  773.00000  662.64000  14.3%  31.7 15827s
 13947668 6498336  718.49990   75   44  773.00000  662.64000  14.3%  31.7 15832s
 13949575 6498989  685.40193   55   49  773.00000  662.64146  14.3%  31.7 15836s
 13953329 6500615  716.37215   67   42  773.00000  662.64500  14.3%  31.7 15841s
 13956967 6502216 infeasible

 14286030 6642203  724.33537   55   40  773.00000  662.91667  14.2%  31.7 16297s
 14287959 6642984  690.24000   59   41  773.00000  662.91893  14.2%  31.7 16300s
 14291624 6644656  699.09166   59   42  773.00000  662.92000  14.2%  31.7 16305s
 14295056 6646124  722.56889   57   38  773.00000  662.92333  14.2%  31.7 16310s
 14298714 6647743  725.37520   60   24  773.00000  662.92667  14.2%  31.7 16316s
 14302522 6649394     cutoff   57       773.00000  662.93000  14.2%  31.7 16321s
 14306120 6650850  677.48481   54   47  773.00000  662.93333  14.2%  31.7 16326s
 14309685 6652450  721.16810   63   26  773.00000  662.93624  14.2%  31.7 16330s
 14312565 6653659  668.83333   56   44  773.00000  662.93917  14.2%  31.7 16335s
 14316427 6655341  766.70000   59   38  773.00000  662.94077  14.2%  31.7 16340s
 14320254 6656868  713.44222   73   60  773.00000  662.94500  14.2%  31.7 16345s
 14325534 6658941  690.09716   58   43  773.00000  662.95000  14.2%  31.7 16352s
 14329175 6660537  664.35593

x(3,15)  =  0.0
x(3,16)  =  0.0
x(3,17)  =  0.0
x(3,18)  =  0.0
x(3,19)  =  0.0
x(3,20)  =  0.0
x(3,21)  =  0.0
x(3,22)  =  0.0
x(3,23)  =  0.0
x(3,24)  =  0.0
x(3,25)  =  0.0
x(3,26)  =  0.0
x(3,27)  =  0.0
x(3,28)  =  0.0
x(3,29)  =  0.0
x(3,30)  =  0.0
x(3,31)  =  0.0
x(3,32)  =  0.0
y(4)  =  28.0
u(4)  =  68.0
v(4)  =  114.0
x(4,0)  =  0.0
x(4,1)  =  0.0
x(4,2)  =  0.0
x(4,3)  =  0.0
x(4,4)  =  0.0
x(4,5)  =  0.0
x(4,6)  =  0.0
x(4,7)  =  0.0
x(4,8)  =  0.0
x(4,9)  =  0.0
x(4,10)  =  0.0
x(4,11)  =  1.0
x(4,12)  =  0.0
x(4,13)  =  0.0
x(4,14)  =  0.0
x(4,15)  =  0.0
x(4,16)  =  0.0
x(4,17)  =  0.0
x(4,18)  =  0.0
x(4,19)  =  0.0
x(4,20)  =  0.0
x(4,21)  =  0.0
x(4,22)  =  0.0
x(4,23)  =  0.0
x(4,24)  =  0.0
x(4,25)  =  0.0
x(4,26)  =  0.0
x(4,27)  =  0.0
x(4,28)  =  0.0
x(4,29)  =  0.0
x(4,30)  =  0.0
x(4,31)  =  0.0
x(4,32)  =  0.0
y(5)  =  9.0
u(5)  =  90.0
v(5)  =  999942.0
x(5,0)  =  0.0
x(5,1)  =  0.0
x(5,2)  =  0.0
x(5,3)  =  0.0
x(5,4)  =  0.0
x(5,5)  =  0.0
x(5,6)  =  0.0
x

x(18,5)  =  0.0
x(18,6)  =  0.0
x(18,7)  =  0.0
x(18,8)  =  1.0
x(18,9)  =  0.0
x(18,10)  =  0.0
x(18,11)  =  0.0
x(18,12)  =  0.0
x(18,13)  =  0.0
x(18,14)  =  0.0
x(18,15)  =  0.0
x(18,16)  =  0.0
x(18,17)  =  0.0
x(18,18)  =  0.0
x(18,19)  =  0.0
x(18,20)  =  0.0
x(18,21)  =  0.0
x(18,22)  =  0.0
x(18,23)  =  0.0
x(18,24)  =  0.0
x(18,25)  =  0.0
x(18,26)  =  0.0
x(18,27)  =  0.0
x(18,28)  =  0.0
x(18,29)  =  0.0
x(18,30)  =  0.0
x(18,31)  =  0.0
x(18,32)  =  0.0
y(19)  =  3.0
u(19)  =  45.0
v(19)  =  999922.0
x(19,0)  =  0.0
x(19,1)  =  0.0
x(19,2)  =  0.0
x(19,3)  =  0.0
x(19,4)  =  0.0
x(19,5)  =  0.0
x(19,6)  =  0.0
x(19,7)  =  0.0
x(19,8)  =  0.0
x(19,9)  =  0.0
x(19,10)  =  0.0
x(19,11)  =  0.0
x(19,12)  =  0.0
x(19,13)  =  0.0
x(19,14)  =  0.0
x(19,15)  =  0.0
x(19,16)  =  0.0
x(19,17)  =  1.0
x(19,18)  =  0.0
x(19,19)  =  0.0
x(19,20)  =  0.0
x(19,21)  =  0.0
x(19,22)  =  0.0
x(19,23)  =  0.0
x(19,24)  =  0.0
x(19,25)  =  0.0
x(19,26)  =  0.0
x(19,27)  =  0.0
x(19,28)  =  0.

x(32,30)  =  1.0
x(32,31)  =  0.0
x(32,32)  =  0.0
